In [91]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from resnet import SimpleResNetModule

tf.executing_eagerly = True

from matplotlib import pyplot as plt
%matplotlib inline

import scipy.stats as stats

In [92]:
class ResNetModel(tf.keras.Model):
    def __init__(self,
                 hparams = [
                    {"filters": 4, "kernel_size": 3, "activation": "relu"},
                    {"filters": 4, "kernel_size": 3, "activation": "relu"},
                 ],
                 name = None):
        super().__init__(name = name)
        self.hparams = hparams
        self.dense = tf.keras.layers.Dense(10, activation = "softmax")
        self.flatten = tf.keras.layers.Flatten()
        self.blocks = [SimpleResNetModule(hparams = block_hparams) for block_hparams in self.hparams]
    
    def call(self, x):
        for block in self.blocks:
            x = block(x)
        return self.dense(self.flatten(x))

In [86]:
model = ResNetModel()
model.compile(
    # By default, fit() uses tf.function().  You can
    # turn that off for debugging, but it is on now.
    run_eagerly=False,

    # Using a built-in optimizer, configuring as an object
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=1e-1),

    # Keras comes with built-in MSE error
    # However, you could use the loss function
    # defined above
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [87]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

model.fit(
    x = x_train,
    y = y_train,
    epochs = 3,
    batch_size = 32,
) 

Epoch 1/3
1875/1875 [==============================] - 22s 12ms/step - loss: 0.2148 - accuracy: 0.9338

In [89]:
print("Test accuracy: %.2f%%"%(tf.keras.metrics.Accuracy()(np.argmax(model(x_test), axis = 1), y_test).numpy()*100))

Test accuracy: 97.95%
